<div class="row">
  <div class="column">
    <img src="./img/logo-onera.png" width="200">
  </div>
  <div class="column">
    <img src="./img/logo-ISAE_SUPAERO.png" width="200">
  </div>
</div>

# FAST-OAD-GA Tutorial

FAST-OAD-GA is an add-on package for [FAST-OAD framework](https://github.com/fast-aircraft-design/FAST-OAD) for performing rapid Overall Aircraft Design in the category General Aviation (GA). The computational core of FAST-OAD being based on the  [OpenMDAO framework](https://openmdao.org/).

In [90]:
import warnings

warnings.filterwarnings(action="ignore")

## 1. Setting up and analyzing the initial problem

To organize our work, we propose to use two user folders `data/` and `workdir/`. For instance, in `data/` we store a XML file which describes the Beechcraft Duchess. In `workdir/`, we store files generated or modified by FAST-OAD.

In [91]:
import os.path as pth
from fastoad import api as api_cs25
import logging
import shutil

from IPython.core.display import display, HTML

# Define relative path
DATA_FOLDER_PATH = "data"
WORK_FOLDER_PATH = "workdir"

# Remove work folder
shutil.rmtree(WORK_FOLDER_PATH, ignore_errors=True)

# Define files
CONFIGURATION_FILE = pth.join(WORK_FOLDER_PATH, "oad_process.yml")
SOURCE_FILE = pth.join(DATA_FOLDER_PATH, "beechcraft_76.xml")

# For having log messages on screen
logging.basicConfig(level=logging.INFO, format="%(levelname)-8s: %(message)s")

# For using all screen width
display(HTML("<style>.container { width:95% !important; }</style>"))

After defining a configuration file name, we can ask FAST-GA to generate a default configuration file based on the default OAD model implemented in the framework:

In [92]:
api_cs25.generate_configuration_file(
    CONFIGURATION_FILE,
    overwrite=True,
    distribution_name="fast-oad-cs23",
    sample_file_name="fastga.yml",
)

WindowsPath('C:/Users/a.carotenuto/Documents/GitHub/FAST-OAD-CS23-HE/FAST-OAD_notebooks/fast-oad-cs23/tutorial/workdir/oad_process.yml')

You can now checkout the generated [configuration file](./workdir/oad_process.yml). In this configuration file, we have specified an input file name 'problem_inputs.xml'. We can ask FAST-OAD to generate the inputs of the default model with the CeRAS parameters as default values:

In [93]:
api_cs25.generate_inputs(CONFIGURATION_FILE, SOURCE_FILE, overwrite=True)

'C:/Users/a.carotenuto/Documents/GitHub/FAST-OAD-CS23-HE/FAST-OAD_notebooks/fast-oad-cs23/tutorial/workdir/problem_inputs.xml'

You can now checkout the generated [input file](./workdir/problem_inputs.xml). As shown previously in the user file architecture, the values in this file can be modified by the user and will be considered by FAST-OAD when executing a computational process.

A useful feature that FAST-OAD provides is to list the outputs of the model defined in the configuration file:

In [46]:
api_cs25.list_variables(CONFIGURATION_FILE)

NAME,I/O,DESCRIPTION
data:TLAR:NPAX_design,IN,"design number of passengers (two pilots are included de facto, meaning for a 2 seater, NPAX_design is equal to 0)"
data:TLAR:luggage_mass_design,IN,luggage design mass
data:TLAR:range,IN,design range
data:TLAR:v_approach,IN,approach speed
data:TLAR:v_cruise,IN,cruise speed
data:TLAR:v_max_sl,IN,maximum speed at sea level
data:aerodynamics:cooling:cruise:CD0,IN,profile drag due to cooling in cruise conditions
data:aerodynamics:cooling:low_speed:CD0,IN,profile drag due to cooling in low speed conditions
data:aerodynamics:propeller:mach,IN,mach number used to compute the polar of the airfoil used in the propeller computation
data:aerodynamics:propeller:reynolds,IN,reynolds number used to compute the polar of the airfoil used in the propeller computation


Another useful feature is to list the modules of the model defined in the configuration file:

In [47]:
api_cs25.list_modules(CONFIGURATION_FILE)

AVAILABLE MODULE IDENTIFIERS,MODULE PATH
fastga.aerodynamics.cl_cd_polar,C:\Users\a.carotenuto\AppData\Local\anaconda3\envs\FAST-OAD-CS23-HE\lib\site-packages\fastga\models\aerodynamics\compute_polar.py
fastga.aerodynamics.highspeed.legacy,C:\Users\a.carotenuto\AppData\Local\anaconda3\envs\FAST-OAD-CS23-HE\lib\site-packages\fastga\models\aerodynamics\aerodynamics_high_speed.py
fastga.aerodynamics.legacy,C:\Users\a.carotenuto\AppData\Local\anaconda3\envs\FAST-OAD-CS23-HE\lib\site-packages\fastga\models\aerodynamics\aerodynamics.py
fastga.aerodynamics.load_factor,C:\Users\a.carotenuto\AppData\Local\anaconda3\envs\FAST-OAD-CS23-HE\lib\site-packages\fastga\models\aerodynamics\load_factor.py
fastga.aerodynamics.lowspeed.legacy,C:\Users\a.carotenuto\AppData\Local\anaconda3\envs\FAST-OAD-CS23-HE\lib\site-packages\fastga\models\aerodynamics\aerodynamics_low_speed.py
fastga.aerodynamics.propeller,C:\Users\a.carotenuto\AppData\Local\anaconda3\envs\FAST-OAD-CS23-HE\lib\site-packages\fastga\models\aerodynamics\external\propeller_code\compute_propeller_aero.py
fastga.aerodynamics.propeller.coeff_map,C:\Users\a.carotenuto\AppData\Local\anaconda3\envs\FAST-OAD-CS23-HE\lib\site-packages\fastga\models\aerodynamics\external\propeller_code\compute_propeller_coefficient_map.py
fastga.aerodynamics.stability_derivatives.legacy,C:\Users\a.carotenuto\AppData\Local\anaconda3\envs\FAST-OAD-CS23-HE\lib\site-packages\fastga\models\aerodynamics\aerodynamics_stability_derivatives.py
fastga.geometry.alternate,C:\Users\a.carotenuto\AppData\Local\anaconda3\envs\FAST-OAD-CS23-HE\lib\site-packages\fastga\models\geometry\geometry.py
fastga.geometry.legacy,C:\Users\a.carotenuto\AppData\Local\anaconda3\envs\FAST-OAD-CS23-HE\lib\site-packages\fastga\models\geometry\geometry.py


Another useful feature is the [N2 diagram](http://openmdao.org/twodocs/versions/latest/basic_guide/make_n2.html) visualization available in OpenMDAO to see the structure of the model:

In [48]:
from IPython.display import IFrame

N2_FILE = pth.join(WORK_FOLDER_PATH, "n2.html")
api_cs25.write_n2(CONFIGURATION_FILE, N2_FILE, overwrite=True)

IFrame(src=N2_FILE, width="100%", height="500px")

Alternatively, you can create a [WhatsOpt](https://github.com/OneraHub/WhatsOpt-Doc#whatsopt-documentation) account to generate the XDSM of the problem. If your account is created, you may uncomment next lines and run them (this should take ~ 1 min):

In [50]:
from IPython.display import IFrame

XDSM_FILE = pth.join(WORK_FOLDER_PATH, "xdsm.html")
api_cs25.write_xdsm(CONFIGURATION_FILE, XDSM_FILE, overwrite=True)

IFrame(src=XDSM_FILE, width="100%", height="500px")

Analysis FASTOADModel pushed


## 2. Running your first MDA

### Beechcraft
Here we run an MDA, that is solving the multidisciplinary couplings using the different nested solvers in the model, without running the optimization problem even if it is defined in the configuration file.

In [94]:
eval_problem = api_cs25.evaluate_problem(CONFIGURATION_FILE, overwrite=True)


aircraft_sizing
NL: NLBGS 1 ; 12528315.4 1
NL: NLBGS 2 ; 2145771.65 0.171273757
NL: NLBGS 3 ; 1078331.97 0.0860715852
NL: NLBGS 4 ; 465041.595 0.0371192439
NL: NLBGS 5 ; 96149.0867 0.0076745423
NL: NLBGS 6 ; 65390.6008 0.00521942485
NL: NLBGS 7 ; 32378.2808 0.00258440817
NL: NLBGS 8 ; 12130.2671 0.000968228101
NL: NLBGS Converged


Let's save these results. We will use them in the next workbook, that shows some post-processing utilities.

In [95]:
OUTPUT_FILE = pth.join(WORK_FOLDER_PATH, "problem_outputs.xml")
Beechcraft_OUTPUT_FILE = pth.join(WORK_FOLDER_PATH, "problem_outputs_Beechcraft_800nm_mda.xml")
shutil.copy(OUTPUT_FILE, Beechcraft_OUTPUT_FILE)

'workdir\\problem_outputs_Beechcraft_800nm_mda.xml'

The `variable-viewer` provides a way to inspect the content of the XML file. The dropdown lists above the table allow to filter the displayed variable.

In [96]:
api_cs25.variable_viewer(OUTPUT_FILE)

In particular, you may inspect the `data:handling_qualities:static_margin` variable.
You will see that its value is equal to the value we set as the target meaning the MDA process converged correctly.

### Beechcraft for 1000 nm range
Here we run an MDA but we change one of the Top Level Aircraft Requirement (TLAR): the range. We choose a 1000 nm range instead of 800 nm for the Beechcraft. For that we use the `VariableViewer` tool on the input file to change the range (do not forget to save!). Just like this:
![variable_viewer](./img/variable_viewer_change_range.gif)

In [99]:
INPUT_FILE = pth.join(WORK_FOLDER_PATH, "problem_inputs.xml")
api_cs25.variable_viewer(INPUT_FILE)

Now that the range has been changed, we run again the MDA.

In [100]:
eval_problem = api_cs25.evaluate_problem(CONFIGURATION_FILE, overwrite=True)


aircraft_sizing
NL: NLBGS 1 ; 12528315.4 1
NL: NLBGS 2 ; 2145771.65 0.171273757
NL: NLBGS 3 ; 1078331.97 0.0860715852
NL: NLBGS 4 ; 465041.595 0.0371192439
NL: NLBGS 5 ; 96149.0867 0.0076745423
NL: NLBGS 6 ; 65390.6008 0.00521942485
NL: NLBGS 7 ; 32378.2808 0.00258440817
NL: NLBGS 8 ; 12130.2671 0.000968228101
NL: NLBGS Converged


Let's save again these new results, for post-processing them in next notebook.

In [101]:
OUTPUT_FILE = pth.join(WORK_FOLDER_PATH, "problem_outputs.xml")
Beechcraft_1000nm_OUTPUT_FILE = pth.join(
    WORK_FOLDER_PATH, "problem_outputs_Beechcraft_1000nm_mda.xml"
)
shutil.copy(OUTPUT_FILE, Beechcraft_1000nm_OUTPUT_FILE)

'workdir\\problem_outputs_Beechcraft_1000nm_mda.xml'

## 3. Running your first MDO

## Beechcraft 800 nm optimization

As seen earlier, the current aircraft configuration is estimated with a fixed distance between the wing aerodynamic center and the horizontal tail aerodynamic center. We will unlock this parameter using a simple optimization problem that will move horizontal tail position in order to minimize the mission consumption. 

The default configuration file defines this optimization problem that aims at:
- minimizing the fuel consumption for the mission (objective),
- with respect to the distance between the wing and the tail aerodynamic center (design variables): by extension the horizontal tail to wing distance,
- subject to no additional constraint.

However, running the MDO with the propeller performance estimation will take a very long time. Consequently, and since the propeller won't change from one MDO loop to another, we decided to use a different .yml file for the probleme definition and reuse the propeller performance map computed in the MDA's hence why the input file will be the output file of a previous MDA.

*(This run should take approximately 20 minutes)*

In [102]:
CONFIGURATION_FILE_MDO = pth.join(WORK_FOLDER_PATH, "oad_process_mdo.yml")
SOURCE_FILE_MDO = pth.join(WORK_FOLDER_PATH, "problem_outputs_Beechcraft_800nm_mda.xml")
shutil.copy(pth.join(DATA_FOLDER_PATH, "fastga_mdo.yml"), CONFIGURATION_FILE_MDO)

'workdir\\oad_process_mdo.yml'

In [103]:
# Set back the inputs from the reference Beechcraft 800 nm
api_cs25.generate_inputs(CONFIGURATION_FILE_MDO, SOURCE_FILE_MDO, overwrite=True)

'C:/Users/a.carotenuto/Documents/GitHub/FAST-OAD-CS23-HE/FAST-OAD_notebooks/fast-oad-cs23/tutorial/workdir/problem_inputs.xml'

To visualize and edit the optimization problem definition (present in the configuration file .toml) you can use the `optimization_viewer` tool. If design variables or constraints have active bounds they are yellow whereas they are red if they are violated. Modifiying the `Initial Value` will modify the input file defined in the configuration file .toml whereas `Value` corresponds to the value found in the output file defined in the configuration file (here it is the 800 nm MDA run).

In [104]:
api_cs25.optimization_viewer(CONFIGURATION_FILE_MDO)

In [105]:
optim_problem = api_cs25.optimize_problem(CONFIGURATION_FILE_MDO, overwrite=True)

NL: NLBGS 1 ; 12528315.4 1
NL: NLBGS 2 ; 2145771.65 0.171273757
NL: NLBGS 3 ; 1078331.97 0.0860715852
NL: NLBGS 4 ; 465041.595 0.0371192439
NL: NLBGS 5 ; 96149.0867 0.0076745423
NL: NLBGS 6 ; 65394.4437 0.00521973158
NL: NLBGS 7 ; 89855.4282 0.00717218757
NL: NLBGS 8 ; 11347.9006 0.000905780241
NL: NLBGS Converged
NL: NLBGS 1 ; 8407.42646 1
NL: NLBGS 2 ; 2589.38995 0.307988415
NL: NLBGS 3 ; 969.233243 0.115282988
NL: NLBGS 4 ; 676.867816 0.0805083244
NL: NLBGS 5 ; 315.027803 0.0374701825
NL: NLBGS 6 ; 195.384896 0.0232395606
NL: NLBGS 7 ; 121.283027 0.0144257018
NL: NLBGS 8 ; 68.9457977 0.00820058291
NL: NLBGS 9 ; 39.2561695 0.00466922544
NL: NLBGS 10 ; 21.9513433 0.00261094681
NL: NLBGS 11 ; 12.0167599 0.00142930301
NL: NLBGS 12 ; 6.51205692 0.000774560082
NL: NLBGS Converged
NL: NLBGS 1 ; 219.440127 1
NL: NLBGS 2 ; 698642.078 3183.74806
NL: NLBGS 3 ; 65918.699 300.394918
NL: NLBGS 4 ; 131272.436 598.215275
NL: NLBGS 5 ; 26034.7385 118.641649
NL: NLBGS 6 ; 16514.998 75.2596994
NL: NLB

Let's save these results:

In [108]:
OUTPUT_FILE = pth.join(WORK_FOLDER_PATH, "problem_outputs.xml")
CeRAS_OPT_OUTPUT_FILE = pth.join(WORK_FOLDER_PATH, "problem_outputs_Beechcraft_800nm_mdo.xml")
shutil.copy(OUTPUT_FILE, CeRAS_OPT_OUTPUT_FILE)

'workdir\\problem_outputs_Beechcraft_800nm_mdo.xml'

The `optimizer_viewer` offers a convenient summary of the optimization result:

In [109]:
api_cs25.optimization_viewer(CONFIGURATION_FILE_MDO)

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\a.carotenuto\\Documents\\GitHub\\FAST-OAD-CS23-HE\\FAST-OAD_notebooks\\fast-oad-cs23\\tutorial\\workdir\\oad_process_mdo.yml'

You can use the `VariableViewer` tool to see the optimization results for all variables of the system by loading the .xml output file:

In [110]:
RESULT_FILE = pth.join(WORK_FOLDER_PATH, "problem_outputs.xml")
api_cs25.variable_viewer(RESULT_FILE)